In [73]:
import numpy as np
import cvxpy as cvx
import time 
import pandas as pd
cost_matrix = np.array([[1000,4,5,1],
                             [4, 1000, 7,3],
                             [5, 7, 1000,5],
                             [1,3,5,1000]])
destinations = ['A', 'B', 'C', 'D']
def visualize(matrix, destinations):
    """
    Visualizes the matrix with the destinations
    """
    df = pd.DataFrame(matrix, columns=destinations, index=destinations)
    print(df)
visualize(cost_matrix, destinations)

      A     B     C     D
A  1000     4     5     1
B     4  1000     7     3
C     5     7  1000     5
D     1     3     5  1000


In [87]:
def get_path(path_matrix, destinations):
    """
    Given the boolean matrix, returns the string that shows the corresponding path,
    assuming the path is a Hamilton path.
    """
    index_sequence = [0]
    n = path_matrix.shape[0]
    current_place = 0
    for i in range(n):
        next_place = int(np.array(range(n)).dot(path_matrix[current_place,:]))
        index_sequence.append(next_place)
        current_place = next_place
    path_str = destinations[0] 
    for i in index_sequence[1:]:
        path_str += ' - ' + destinations[i]
    return path_str
# demo
path_matrix = np.array([[0., 0., 0., 1.],
               [0., 0., 1., 0.],
               [1., 0., 0., 0.],
               [0., 1., 0., 0.]])
print('The path matrix is:\n', path_matrix)
print('The corresponding path is:')
print(get_path(path_matrix, destinations))

The path matrix is:
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
The corresponding path is:
A - D - B - C - A


In [96]:
def tsp_tmz(cost_matrix):
    """
    Solves the TSP with TMZ constraints. 
    
    Inputs:
    - cost_matrix: 2D array that represents the cost of moving from one place to another
    
    Outputs: 
    - path: 2D array, the optimal path matrix, with binary values. Path[i,j]=1 means
    we go from location i to location j
    - optim_value: float, the total time travelled (the optimal cost)
    - duration: float, time taken (in seconds) to run the algorithm 
    """
    # number of destinations
    n = cost_matrix.shape[0]
    # path matrix variable
    x = cvx.Variable((n,n), boolean=True)
    # sum constraints: sum of each row and column must be 1
    con = [cp.sum(x, axis=0) == np.ones(n),
           cp.sum(x, axis=1) == np.ones(n)]
    
    # extra variables in MTZ formulation
    u = cvx.Variable(n)

    # Condition set 1. u_0 = 0
    con.append(u[0] == 0)
    # Consition set 2. 1 <= u_i <= n-1
    con += [1 <= u[i] for i in range(1,n)]
    con += [u[i] <= n-1 for i in range(1,n)]
    # Condition set 3. u_i - u_j + 1 <= (n-1)(1-x_ij)
    con += [u[i] - u[j] + 1 <= (n-1)*(1-x[i,j]) for i in range(1,n) for j in range(1,n)]

    # objective function: the dot product between the cost matrix and the path matrix 
    # Here we use .multiply along with .sum to calculate the dot product 
    obj = cvx.Minimize(cvx.sum(cvx.multiply(cost_matrix,x)))

    # solve the problem 
    prob = cvx.Problem(obj, con)
    # timing the running time
    start = time.time()
    prob.solve()  # Returns the optimal value.
    duration = time.time() - start
    
    # rounding the values in the optimal path matrix
    path = np.around(x.value)
    
    # the cost (total time travelled)
    optim_value = prob.value
    print('Total optimal cost:', optim_value)
    
    # print nicely the path matrix
    visualize(path, destinations)
    print(u.value)
    return path, optim_value, duration

path_matrix, optim_cost, t = tsp_tmz(cost_matrix)

# print out the path we need to take
print('\nOptimal path:')
print(get_path(path_matrix, destinations))


Total optimal cost: 15.999999995728647
     A    B    C    D
A -0.0  0.0 -0.0  1.0
B  0.0 -0.0  1.0  0.0
C  1.0  0.0 -0.0 -0.0
D -0.0  1.0  0.0 -0.0
[0. 2. 3. 1.]

Optimal path:
A - D - B - C - A
